# Generador de preguntas de Moodle

In [19]:
# Paquetes
import itertools
import os
import random

## Parámetros y enunciado

Se deben completar los siguientes campos para la generación de las preguntas del cuestionario.

1. **Nombre del cuestionario:** Asigne un nombre descriptivo al cuestionario.
2. **Parámetros:** Enumere todos los parámetros utilizados en el la pregunta.
3. **Diccionario de parámetros:** Proporcione un diccionario con los parámetros y listas de valores de cada uno.
4. **Enunciado:** Escriba el enunciado de la pregunta. Utilice doble corchete `[[ ]]` para encerrar los parámetros o cálculos que desee que el generador reemplace.
    - Si un item dentro del enunciado no tiene opciones, use corchetes vacíos `[]`.
    - Incluya siempre el parámetro `[[id]]` en cada pregunta para su reemplazo automático.
5. **Cantidad máxima de ejercicios:** Especifique el número máximo de preguntas que desea que se generen para este cuestionario.
6. **Funciones (opcional):** Código adicional a compilar antes de la generación de cada pregunta, se debe colocar en formato string (entre comillas).
7. **Plantilla (opcional):** Nombre del archivo .tex que será usado como plantilla, de no especificar, se utilizará el archivo `Plantilla.tex`.
8. **Salida (opcional):** Nombre de la carpeta de salida, de no especificar, seutilizará `output`.

In [20]:
# Nombre del cuestionario
name = "Producto"

# Parámetros
par = ["a", "b"]
# Diccionario de parámetros
par_dict = {
    "a": [-2,-1,0,1,2],
    "b": [-2,-1,1,2],
    }

# Enunciado
enunciado = r'''
\begin{numerical}[]%
    % - Indentificador
    {Producto - [[id]]}
    % - Enunciado
    Determine el valor de
    \[
        ([[a]]) \cdot ([[b]])
    \]
    \item[] [[a*b]]
\end{numerical}
'''

# Cantidad de preguntas a generar
n_preguntas = 2

# Código adicional a compilar antes de la generación de cada pregunta (opcional)
# funciones = []
# Plantilla personalizada (opcional, por defento Plantilla.tex)
# plantilla = "PlantillaAleph.tex"
# Carpeta de salida (opcional, por defecto output/)
# carpeta = "output/"

## Generar preguntas

Ejecutar la siguiente celda, se obtendrá el código en LaTeX de las perguntas generadas.


In [21]:
# Total preguntas
_total = n_preguntas
# Reemplazo todos los { por {{
_enunciado_F = enunciado.replace('{', '{{')
_enunciado_F = _enunciado_F.replace('}', '}}')
# Reemplazo todos los corchetes por {
_enunciado_F = _enunciado_F.replace('[[', '{')
_enunciado_F = _enunciado_F.replace(']]', '}')

# Lista de parámetros
_par_list = []
for _parametro in par:
    _par_list.append(par_dict[_parametro])

# Semilla
random.seed(18)

# Producto cartesiano
_par_comb = list(itertools.product(*_par_list))
_par_comb = random.sample(_par_comb, _total)

# Genero el examen
_quiz = ''
id = 1
if not 'funciones' in locals():
    funciones = []
for _parametros in _par_comb:
    try:
        for _n, _parametro in enumerate(par):
            exec(f"{_parametro} = _parametros[{_n}]")
        for _funcion in funciones:
            exec(_funcion)
        # Reemplazo
        _quiz += '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
        exec(f'_quiz += fr"""{_enunciado_F}"""')
        _quiz += '\n'
        id += 1
    except:
        print('Error en los parámetros:',par, '=', _parametros)
    if id > _total:
        break

# Mensaje
print(_quiz)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[]%
    % - Indentificador
    {Producto - 1}
    % - Enunciado
    Determine el valor de
    \[
        (-1) \cdot (-1)
    \]
    \item[] 1
\end{numerical}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
\begin{numerical}[]%
    % - Indentificador
    {Producto - 2}
    % - Enunciado
    Determine el valor de
    \[
        (-2) \cdot (2)
    \]
    \item[] -4
\end{numerical}




## Generar .tex, .pdf y .xml

Al ejecutar la siguiente celda, se obtendrá el archivo .tex, .pdf y .xml del cuestionario generado.

In [22]:
# Total preguntas
_total = n_preguntas
# Reemplazo todos los { por {{
_enunciado_F = enunciado.replace('{', '{{')
_enunciado_F = _enunciado_F.replace('}', '}}')
# Reemplazo todos los corchetes por {
_enunciado_F = _enunciado_F.replace('[[', '{')
_enunciado_F = _enunciado_F.replace(']]', '}')

# Lista de parámetros
_par_list = []
for _parametro in par:
    _par_list.append(par_dict[_parametro])

# Producto cartesiano
_par_comb = list(itertools.product(*_par_list))

# Semilla
random.seed(18)

# Si está definido el máximo de ejercicios
if n_preguntas:
    # Tomo combinaciones aleatorias
    _par_comb = random.sample(_par_comb, n_preguntas)

# Genero el examen
_quiz = ''
id = 1
if not 'funciones' in locals():
    funciones = []
for _parametros in _par_comb:
    try:
        for _n, _parametro in enumerate(par):
            exec(f"{_parametro} = _parametros[{_n}]")
        for _funcion in funciones:
            exec(_funcion)
        # Reemplazo
        _quiz += '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
        exec(f'_quiz += fr"""{_enunciado_F}"""')
        _quiz += '\n'
        id += 1
    except:
        print('Error en los parámetros:',par, '=', _parametros)
    if id > _total:
        break

_parametros_latex = ''
for _parametro in par:
    _parametros_latex += '\t' + fr'\item ${_parametro} \in ' + '\\{' + ', '.join(map(str, par_dict[_parametro])) + '\\}$' + '\n'

# Leo el archivo plantilla
# Si la variable plantilla no está definida, la defino como 'Plantilla.tex'
if not 'plantilla' in locals():
    plantilla = 'Plantilla.tex'
_plantilla = open(plantilla, 'r', encoding='utf-8')
_t_plantilla = _plantilla.read()
_plantilla.close()
# Reemplazo
_t_plantilla = _t_plantilla.replace('{{QUIZ}}', _quiz)
_t_plantilla = _t_plantilla.replace('{{Cuestionario}}', name)
_t_plantilla = _t_plantilla.replace('{{Parámetros}}', _parametros_latex)
_t_plantilla = _t_plantilla.replace('{{Número de preguntas}}', str(id-1))

# Escribo el archivo
# Si la variable carpeta no está definida, la defino como 'test'
if not 'carpeta' in locals():
    carpeta = 'output'
# Si no existe la carpeta, la creo
if not os.path.exists(carpeta):
    os.makedirs(carpeta)
_examen = open(f'{carpeta}/{name}.tex', 'w', encoding='utf-8')
_examen.write(_t_plantilla)
_examen.close()

# Compilo el examen con xeLaTeX
os.system(f'latexmk -xelatex "{name}.tex" -output-directory="./{carpeta}"')

# Elimino todos los archivos llamados examen, salvo el pdf y el xml
_archivos = os.listdir(f'./{carpeta}')
for _archivo in _archivos:
    if name in _archivo:
        if '.pdf' not in _archivo and '.xml' not in _archivo and '.tex' not in _archivo:
            try:
                os.remove(f"{carpeta}/"+_archivo)
            except:
                pass
# Elimino los archivos fls
for _archivo in _archivos:
    if '.fls' in _archivo:
        try:
            os.remove(_archivo)
        except:
            pass

# Mensaje
print(f'Examen generado con éxito, se generaron {id-1} preguntas.')

Examen generado con éxito, se generaron 2 preguntas.
